In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/tf/notebooks/src


## Intialization

### GPU

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Packages

In [4]:
!pip install pandas iterative-stratification tensorflow_addons nlpaug==0.0.20 tqdm click

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### Imports

In [5]:
import os
import random
import logging
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa


from dataloader import BalancedMelSampler, MelSampler, convert_csv_to_dict_for_dataloader
from losses import NpairsLoss
from metrics import TFLWLRAP
from models import NUM_FRAMES, Classifier, DeepMetricLearning
from split_data import get_split
from train import get_model, get_callbacks, get_lr_metric

In [6]:
from params import *
from utils.logger import prepare_log_folder, create_logger

### Setup

In [7]:
warnings.filterwarnings("ignore")
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["SM_FRAMEWORK"] = "tf.keras"

physical_devices = tf.config.list_physical_devices("GPU")
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Main

In [8]:
def main(fold_idx, saved_path, pretrained_path, pretrained_with_contrastive):
    os.makedirs(os.path.join(saved_path, f"fold{fold_idx}"), exist_ok=True)
    pretrained_with_contrastive = bool(pretrained_with_contrastive)

    print(' -> Preparing Data \n')
    
    train_data = pd.read_csv("../data/new_train_tp.csv")
    train_index, val_index = get_split(fold=fold_idx)
    fold_train_dict = convert_csv_to_dict_for_dataloader(train_data.iloc[train_index])
    fold_valid_dict = convert_csv_to_dict_for_dataloader(train_data.iloc[val_index])

    balanced_train_data_loader = BalancedMelSampler(
        fold_train_dict,
        batch_size=64,
        max_length=NUM_FRAMES,
        is_train=True,
        n_classes=24,
        use_cutmix=True,
        cache=True,
        n_classes_in_batch=8,
        shuffle_aug=False,
    )

    valid_data_loader = MelSampler(
        fold_valid_dict,
        batch_size=balanced_train_data_loader.batch_size,
        n_classes=balanced_train_data_loader.n_classes,
        cache=True,
        max_length=NUM_FRAMES,
        is_train=False,
        use_cutmix=False,
        shuffle_aug=balanced_train_data_loader.shuffle_aug,
    )

    print(' -> Preparing Model \n')
    model = get_model(
        saved_path=saved_path,
        pretrained_with_contrastive=pretrained_with_contrastive,
        pretrained_path=pretrained_path,
    )
    model._build()
    
    scheduler = tfa.optimizers.Triangular2CyclicalLearningRate(
        initial_learning_rate=5e-5,
        maximal_learning_rate=1e-3,
        step_size=15 * 30,
    )
    
    optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(
        tfa.optimizers.Lookahead(
            tf.keras.optimizers.Adam(learning_rate=scheduler),
            10,
            0.5,
        ),
        "dynamic",
    )
    
    model.compile(
        optimizer=optimizer,
        metrics=[TFLWLRAP(num_classes=24, name="lwlrap"), get_lr_metric(optimizer)],
        metric_loss_fn=NpairsLoss(temperature=0.1, name="n_pairs"),
        classification_loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    )

#     model.summary()
#     return 0

    print(' -> Training Model \n')

    callbacks = get_callbacks(pretrained_with_contrastive, fold_idx, saved_path=saved_path)
    steps_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    
    model.fit(
        balanced_train_data_loader,
        steps_per_epoch=steps_per_epoch,
        epochs=100,
        validation_data=valid_data_loader,
        callbacks=callbacks,
        verbose=2
    )

In [11]:
DEBUG = False
pretrained_folder = "../logs/2021-01-28/1/"
log_folder = "../logs/"

In [ ]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'Logging results to {log_folder}')
    create_logger(directory=log_folder, name="logs.txt")
    
for fold_idx in range(5):
    print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")
    
    main(
        fold_idx, 
        log_folder, 
        pretrained_folder + f"pretrained_best_fold{fold_idx}.h5", 
        pretrained_with_contrastive=False
    )
    
    if DEBUG:
        break

Logging results to ../logs/2021-01-28/11/

-------------   Fold 1 / 5  -------------

 -> Preparing Data 

 -> Preparing Model 

 -> Loading weights from ../logs/2021-01-28/1/pretrained_best_fold0.h5

 -> Training Model 

Epoch 1/100
Instructions for updating:
Use fn_output_signature instead
15/15 - 8s - loss: 0.1731 - lwlrap: 0.2387 - val_loss: 0.1799 - val_lwlrap: 0.3126
Epoch 2/100
15/15 - 6s - loss: 0.1415 - lwlrap: 0.4427 - val_loss: 0.1775 - val_lwlrap: 0.4629
Epoch 3/100
15/15 - 6s - loss: 0.1525 - lwlrap: 0.5908 - val_loss: 0.1636 - val_lwlrap: 0.5416
Epoch 4/100
15/15 - 6s - loss: 0.2048 - lwlrap: 0.6641 - val_loss: 0.1474 - val_lwlrap: 0.6033
Epoch 5/100
15/15 - 6s - loss: 0.0989 - lwlrap: 0.7459 - val_loss: 0.1342 - val_lwlrap: 0.6584
Epoch 6/100
